In [1]:
# import packages
# pip install selenium 
# pip install beautifulsoup4

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
import requests

import time, datetime
import pandas as pd
import numpy as np
import math, re, sys
import warnings
warnings.filterwarnings("ignore")

In [2]:
def scroll_to_bottom(driver,sleep_time=120):
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            break
        last_height = new_height
    
    time.sleep(sleep_time)  

In [3]:
def find_Job_Ids(soup):

    Job_Ids_on_the_page = []
    
    job_postings = soup.find_all('li', {'class': 'jobs-search-results__list-item'})
    for job_posting in job_postings:
        Job_ID = job_posting.get('data-occludable-job-id')
        Job_Ids_on_the_page.append(Job_ID)
        # job_title = job_posting.find('a', class_='job-card-list__title').get_text().strip()
        # location = job_posting.find('li', class_='job-card-container__metadata-item').get_text().strip()
    
    return Job_Ids_on_the_page 

In [4]:
def remove_tags(html):
    '''remove html tags from BeautifulSoup.text'''
 
    # parse html content
    soup = BeautifulSoup(html, "html.parser")
 
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
 
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

In [5]:
def clean_Job_description(text):
    senetences_to_remove = ["Remove photo First name Last name Email Password (8+ characters) ",
                            "By clicking Agree & Join",
                            "you agree to the LinkedIn User Agreement",
                            "Privacy Policy and Cookie Policy",
                            "Continue Agree & Join or Apply on company website",
                            "Security verification",
                            "Close Already on LinkedIn ?",
                            "Close Already on LinkedIn?",
                            "Sign in Save Save job Save this job with your existing LinkedIn profile , or create a new one",
                            "Sign in Save Save job Save this job with your existing LinkedIn profile, or create a new one",
                            "Your job seeking activity is only visible to you",
                            "Email Continue Welcome back"]
    for sentence in senetences_to_remove:
        result = text.find(sentence)
        if result>-1:
            text = text[:result] + text[result+len(sentence):] # remove sentence from text

    return text 

In [6]:
def get_posted_date(posted_time_ago,date_scraping):
    """Convert posted_time_ago to number of days.
    For example, 1 month ago is replaced by 30. 1 week by 7 and so on..."""
    posted_date = None
    
    try:
        details = posted_time_ago.split()
        N_DAYS_AGO = int(details[0])
        day_week_month_year = details[1] 
        if day_week_month_year.startswith("day"):
            N_DAYS_AGO = N_DAYS_AGO
        elif day_week_month_year.startswith("week"):
            N_DAYS_AGO = N_DAYS_AGO*7
        elif day_week_month_year.startswith("month"):
            N_DAYS_AGO = N_DAYS_AGO*30
        elif day_week_month_year.startswith("year"):
            N_DAYS_AGO = N_DAYS_AGO*365
        else:
            N_DAYS_AGO = None

        posted_date = date_scraping - datetime.timedelta(days=N_DAYS_AGO)
    except:
        posted_date = None

    return posted_date

In [7]:
def linkedin_scrapper(keyword, user_credentials_path='/Users/Jason/Desktop/is3107/is3107_DataEngineering/user_credentials.txt', chromedriver_path="C:/Users/Jason/Desktop/IS3107/chromedriver.exe"):
    # Get User Credentials
    with open(user_credentials_path, 'r',encoding="utf-8") as file:
        user_credentials = file.readlines()
        user_credentials = [line.rstrip() for line in user_credentials]
    my_email,my_pwd = user_credentials[0],user_credentials[1]

    # 1. Instanciate the chrome service
    chromedriver_path = chromedriver_path
    service = Service(executable_path=chromedriver_path)

    # 2. Instanciate the webdriver
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options, service=service)

    # 3. Open the LinkedIn login page
    driver.get('https://www.linkedin.com/login')
    time.sleep(10) # waiting for the page to load

    # 4. Enter email address & password
    email_input = driver.find_element(By.ID, 'username')
    password_input = driver.find_element(By.ID, 'password')
    email_input.send_keys(my_email)
    password_input.send_keys(my_pwd)

    # 5. Click the login button
    password_input.send_keys(Keys.ENTER)
    time.sleep(60)

    List_Job_IDs = []
    keywords = keyword
    location = 'singapore'
    start = 0
    url = f'https://www.linkedin.com/jobs/search/?keywords={keywords}&location={location}&start={start}'
    url = requests.utils.requote_uri(url)
    driver.get(url)
    scroll_to_bottom(driver,sleep_time=10)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        div_number_of_jobs = soup.find("div",{"class":"jobs-search-results-list__subtitle"})
        number_of_jobs = int(div_number_of_jobs.find('span').get_text().strip().split()[0])
    except:
        number_of_jobs = 0
    number_of_pages=math.ceil(number_of_jobs/25)
    print("number_of_jobs:",number_of_jobs)
    print("number_of_pages:",number_of_pages)

    # Get Job Ids present on the first page.
    Jobs_on_this_page = find_Job_Ids(soup)
    List_Job_IDs.extend(Jobs_on_this_page)

    if number_of_pages>1:
        for page_num in range(1,number_of_pages):
            print(f"Scraping page: {page_num}",end="...")
            
            # Navigate to page
            url = f'https://www.linkedin.com/jobs/search/?keywords={keywords}&location={location}&start={25 * page_num}'
            url = requests.utils.requote_uri(url)
            driver.get(url)
            scroll_to_bottom(driver,sleep_time=5)
            
            # Parse the HTML content of the page using BeautifulSoup.
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Get Job Ids present on the page.
            Jobs_on_this_page = find_Job_Ids(soup)
            List_Job_IDs.extend(Jobs_on_this_page)  
            print(f'Jobs found:{len(Jobs_on_this_page)}')

    ## Close the browser and shut down the ChromiumDriver executable that
    # is started when starting the ChromiumDriver. 
    driver.quit()

    job_url='https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'
    job={}
    list_jobs=[]

    for j in range(0,len(List_Job_IDs)):
        print(f"{j+1} ... read jobId:{List_Job_IDs[j]}")

        resp = requests.get(job_url.format(List_Job_IDs[j]))
        soup=BeautifulSoup(resp.content,'html.parser')

        job["Job_ID"] = List_Job_IDs[j]

        try:
            # apply_link = soup.find("a", {'class':"jobs-apply-button"})["href"]
            apply_link = 'https://www.linkedin.com/jobs/collections/recommended/?currentJobId={}'.format(List_Job_IDs[j])
            # apply_link = job_url.format(List_Job_IDs[j])
            job['url'] = apply_link
        except:
            job['url'] = None

        try: # remove tags
            job["Job_txt"] = remove_tags(resp.content)
            # job["Job_txt"] = soup.find("div", {"class":"description__text description__text--rich"}).text.strip()
        except:
            job["Job_txt"] = None
            
        try:
            job["company"]=soup.find("div",{"class":"top-card-layout__card"}).find("a").find("img").get('alt')
        except:
            job["company"]=None
            
        try:
            job["job-title"]=soup.find("div",{"class":"top-card-layout__entity-info"}).find("a").text.strip()
        except:
            job["job-title"]=None
            
        try:
            job["level"]=soup.find("ul",{"class":"description__job-criteria-list"}).find("li").text.replace("Seniority level","").strip()
        except:
            job["level"]=None
        
        try:
            job["location"]=soup.find("span",{"class":"topcard__flavor topcard__flavor--bullet"}).text.strip()
        except:
            job["location"]=None
            
        try:
            job["posted-time-ago"]=soup.find("span",{"class":"posted-time-ago__text topcard__flavor--metadata"}).text.strip()
        except:
            job["posted-time-ago"]=None
            
        try:
            nb_candidats = soup.find("span",{"class":"num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip()
            nb_candidats = int(nb_candidats.split()[0])
            job["nb_candidats"]= nb_candidats
        except:
            job["nb_candidats"]=None
            
        list_jobs.append(job)
        job={}   

    # create a pandas Datadrame
    jobs_DF = pd.DataFrame(list_jobs) 

    jobs_DF['scraping_date'] = pd.to_datetime(datetime.date.today())
    jobs_DF['posted_date'] = np.vectorize(get_posted_date)(jobs_DF['posted-time-ago'], jobs_DF['scraping_date'])

    jobs_DF['Job_txt'] = jobs_DF['Job_txt'].apply(clean_Job_description)
    jobs_DF.level = jobs_DF.level.apply(lambda x:x.replace("Employment type\n        \n\n          ","") if x is not None else x)

    # jobs_DF.to_json("{}_scraped.json".format(keyword.replace('%20', '')))
    jobs_DF.to_csv("{}_scraped.csv".format(keyword.replace('%20', '')))
    return jobs_DF

In [14]:
# define your user credential path 
user_credentials_path='/Users/Jason/Desktop/is3107/is3107_DataEngineering/user_credentials.txt'

# define your chromedriver path for selenium 
chromedriver_path="C:/Users/Jason/Desktop/IS3107/chromedriver.exe"

# scrape data
linkedin_scrapper('software%20engineer', user_credentials_path, chromedriver_path)
time.sleep(15)
linkedin_scrapper('data%20analyst', user_credentials_path, chromedriver_path)
time.sleep(15)
linkedin_scrapper('data%20scientist', user_credentials_path, chromedriver_path)
time.sleep(15)
linkedin_scrapper('machine%20learning%20engineer', user_credentials_path, chromedriver_path)
time.sleep(15)
linkedin_scrapper('AI%20developer', user_credentials_path, chromedriver_path)
time.sleep(15)
linkedin_scrapper('data%20engineer', user_credentials_path, chromedriver_path)

number_of_jobs: 438
number_of_pages: 18
Scraping page: 1...Jobs found:25
Scraping page: 2...Jobs found:25
Scraping page: 3...Jobs found:25
Scraping page: 4...Jobs found:25
Scraping page: 5...Jobs found:25
Scraping page: 6...Jobs found:25
Scraping page: 7...Jobs found:25
Scraping page: 8...Jobs found:25
Scraping page: 9...Jobs found:25
Scraping page: 10...Jobs found:25
Scraping page: 11...Jobs found:25
Scraping page: 12...Jobs found:25
Scraping page: 13...Jobs found:25
Scraping page: 14...Jobs found:25
Scraping page: 15...Jobs found:25
Scraping page: 16...Jobs found:25
Scraping page: 17...Jobs found:25
1 ... read jobId:3900988093
2 ... read jobId:3908056836
3 ... read jobId:3884004702
4 ... read jobId:3809562317
5 ... read jobId:3630973204
6 ... read jobId:3888556806
7 ... read jobId:3879356294
8 ... read jobId:3883819689
9 ... read jobId:3909568923
10 ... read jobId:3891873342
11 ... read jobId:3866567318
12 ... read jobId:3674173411
13 ... read jobId:3869439550
14 ... read jobId:37388

,Job_ID,url,Job_txt,company,job-title,level,location,posted-time-ago,nb_candidats,scraping_date,posted_date
0,3900988093,https://www.linkedin.com/jobs/collections/reco...,"Data Engineer Avensys Consulting Singapore, Si...",Avensys Consulting,Data Engineer,Mid-Senior level,"Singapore, Singapore",1 week ago,NaN,2024-04-26,2024-04-19
1,3908056836,https://www.linkedin.com/jobs/collections/reco...,"Data Engineer Astek Singapore, Singapore 2 day...",Astek,Data Engineer,Mid-Senior level,"Singapore, Singapore",2 days ago,184.0,2024-04-26,2024-04-24
2,3884004702,https://www.linkedin.com/jobs/collections/reco...,Data Engineer Intern - 6 months Gunvor Group L...,Gunvor Group Ltd,Data Engineer Intern - 6 months,Internship,Singapore,2 weeks ago,88.0,2024-04-26,2024-04-12
3,3809562317,https://www.linkedin.com/jobs/collections/reco...,,None,None,None,None,None,NaN,2024-04-26,NaT
4,3630973204,https://www.linkedin.com/jobs/collections/reco...,"Data Engineer JAC Recruitment Singapore, Singa...",JAC Recruitment,Data Engineer,Entry level,"Singapore, Singapore",1 month ago,NaN,2024-04-26,2024-03-27
...,...,...,...,...,...,...,...,...,...,...,...
445,3774464179,https://www.linkedin.com/jobs/collections/reco...,Database Engineer OKX Singapore 2 months ago 1...,OKX,Database Engineer,Mid-Senior level,Singapore,2 months ago,185.0,2024-04-26,2024-02-26
446,3903501263,https://www.linkedin.com/jobs/collections/reco...,AI Engineer Braintrust Singapore 1 week ago 92...,Braintrust,AI Engineer,Entry level,Singapore,1 week ago,92.0,2024-04-26,2024-04-19
447,3899096084,https://www.linkedin.com/jobs/collections/reco...,"Python Developer Tech Mahindra Singapore, Sing...",Tech Mahindra,Python Developer,Mid-Senior level,"Singapore, Singapore",1 week ago,NaN,2024-04-26,2024-04-19
448,3888585441,https://www.linkedin.com/jobs/collections/reco...,"Machine Learning Engineer Singtel Singapore, S...",Singtel,Machine Learning Engineer,Entry level,"Singapore, Singapore",2 weeks ago,143.0,2024-04-26,2024-04-12
